In [2]:
import tensorflow as tf
from glob import glob
import os
import cv2
from PIL import Image
from tensorflow.keras.utils import normalize
import numpy as np



In [3]:
img_dataset = []
msk_dataset = []

f_img = sorted(glob('dataset/train/asagao/*'))
for f in f_img:
    image = cv2.imread(f)
    # image = image.resize((SIZE, SIZE))
    #Nomalize an image from 0-255 to 0 - 1
    image = image.astype('float32') / 255.0
    img_dataset.append(np.array(image))

f_msk = sorted(glob('dataset/validation/asagao/*'))
for f in f_msk:
    image = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    (thresh, im_bw) = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    image = cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY)[1]
    # image = Image.fromarray(image)
    # image = image.resize((SIZE, SIZE))
    msk_dataset.append(np.array(image))
asagao_dataset = np.array(img_dataset)
asagao_msk_dataset = np.expand_dims(np.array(msk_dataset) /255., -1)

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
X_train, X_test, y_train, y_test = train_test_split(asagao_dataset, asagao_msk_dataset, test_size= 0.10, random_state=0)

### train with asagao pixel

In [7]:
# history_previous = history
history = model.fit(X_train, 
                    y_train,
                    verbose = 1,
                    batch_size = 32,
                    epochs = 100,
                    validation_data = (X_test, y_test),
                    shuffle = False)
model.save("50m_30m_Unet")



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

history = history_previous

acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'b', label='Training acc',linewidth=2)
plt.plot(epochs, val_acc, 'r--', label='Validation acc',linewidth=2)
plt.title('Training  accuracy')
plt.legend()

plt.subplot(1, 2, 2)

plt.plot(epochs, loss, 'b', label='Training loss')

plt.plot(epochs, val_loss, 'k', label='Validationloss ')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
##################################
#IOU
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.6

intersection = np.logical_and(y_test, y_pred_thresholded)
union = np.logical_or(y_test, y_pred_thresholded)
iou_score = np.sum(intersection) / np.sum(union)
print("IoU socre is: ", iou_score)

### Test within known data

In [10]:
def predict_tile(tile, sens = 0.001):
    tile = tile.astype('float32') / 255.0
    test_img_other_input=np.expand_dims(tile, 0)
    prob_map = (model.predict(test_img_other_input) > sensitivty).astype(np.uint8)
    return prob_map[0]
    

# file_name = "asagao_0001.png"
# train_file = 'dataset/train/asagao/' + file_name
# test_file = 'dataset/validation/asagao/' + file_name.replace("asagao_", "asagao_mask_" )

# print(train_file, test_file)
# sensitivty = 0.4
# test_img_other = cv2.imread(train_file)
# r = predict_tile(test_img_other, sensitivty)

# plt.figure(figsize=(20,20))
# actual = cv2.imread(train_file)
# mask = cv2.imread(test_file)
# plt.subplot(131),plt.imshow(actual),plt.title("Actual")
# plt.subplot(132),plt.imshow(mask),plt.title("Actual mask")
# plt.subplot(133),plt.imshow(r, cmap="gray"),plt.title("Prediction")


### Predict with tiled image

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('50m_30m_Unet')
sensitivity = 0.3

In [47]:
import os
import os
from tiled_detiled import tiled_image
import time
from glob import glob


sensitivty = 0.5
target_f = "dataset/raw/labeled/10m/DJI_0007.jpg"

if len(os.listdir('dataset/test/test1')) != 0:
    print("Found previous tiled images, attemp delete")
    file_to_delete = os.listdir('dataset/test/test1')
    for f in file_to_delete:
        os.remove('dataset/test/test1/' + f)

target_im = cv2.imread(target_f)
#save otuput
from glob import glob
import math
result_heigth = target_im.shape[0]
result_width = target_im.shape[1]
tile_size = 256

if target_im is not None:
    tiled_image(target_im, 256, "dataset/test/test1")
else:
    print("Cannot read ifle")


#Slice image
predict_file = sorted(glob("dataset/test/test1/*.png"))
predict_file = iter(predict_file)
# print(predict_file)

predicted_list = []

#remove existing file
if len(os.listdir('dataset/test/result_test1')) != 0:
    print("Found previous predicted files, attemp delete")
    file_to_delete = os.listdir('dataset/test/result_test1')
    for f in file_to_delete:
        os.remove('dataset/test/result_test1/' + f)
#predict slided image
nn = 1
for f in predict_file:
    target = f
    # print(target)
    inputs = cv2.imread(target)
    tic = time.time()
    r = predict_tile(inputs, sensitivity)
    toc = time.time()
    print("predicted %s in %.8f s" %(f, toc - tic))
    tf.keras.preprocessing.image.save_img('dataset/test/result_test1/result_%s.png'% str(nn).zfill(4),r)
    nn += 1



file = sorted(glob("dataset/test/result_test1/*"))

temp = np.zeros((math.ceil(result_heigth / tile_size) * tile_size, math.ceil(result_width / tile_size) * tile_size,3))

i = 0
for r in range(0, temp.shape[0], tile_size):
    for c in range(0, temp.shape[1], tile_size):
        tile = cv2.imread(file[i])
        temp[r:r + tile_size, c:c + tile_size,:] = tile 
        i += 1

cv2.imwrite("predicted_location.jpg" , temp[:result_heigth, :5274,:])
# cv2.imwrite("predicted_location_DJI_20211208095147_0040.JPG" , temp[:5460, :8192,:])


Found previous tiled images, attemp delete
Found previous predicted files, attemp delete
predicted dataset/test/test1/Tile_0001.png in 0.06397486 s
predicted dataset/test/test1/Tile_0002.png in 0.08188486 s
predicted dataset/test/test1/Tile_0003.png in 0.07268691 s
predicted dataset/test/test1/Tile_0004.png in 0.07395077 s
predicted dataset/test/test1/Tile_0005.png in 0.05598974 s
predicted dataset/test/test1/Tile_0006.png in 0.05110860 s
predicted dataset/test/test1/Tile_0007.png in 0.09715867 s
predicted dataset/test/test1/Tile_0008.png in 0.06625485 s
predicted dataset/test/test1/Tile_0009.png in 0.06606221 s
predicted dataset/test/test1/Tile_0010.png in 0.10850334 s
predicted dataset/test/test1/Tile_0011.png in 0.06939650 s
predicted dataset/test/test1/Tile_0012.png in 0.09013867 s
predicted dataset/test/test1/Tile_0013.png in 0.05381894 s
predicted dataset/test/test1/Tile_0014.png in 0.07375765 s
predicted dataset/test/test1/Tile_0015.png in 0.07320166 s
predicted dataset/test/tes

True

In [48]:
import cv2
from PIL import Image
import numpy as np

def convert_bw(img):
    grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bw = cv2.threshold(grayImage, 128, 255, cv2.THRESH_BINARY)[1]
    image_bw = Image.fromarray(bw)
    return image_bw

_ground_truth = cv2.imread('DJI_0007_mask.jpg')
_pred = cv2.imread("predicted_location.jpg")
gt = convert_bw(_ground_truth)
pd = convert_bw(_pred)

print(_ground_truth.shape, _pred.shape)
intersection = np.logical_and(gt, pd)
union = np.logical_or(gt, pd)
iou_score = np.sum(intersection) / np.sum(union)
print("IoU socre is: ", iou_score)



(3956, 5274, 3) (3956, 5274, 3)
IoU socre is:  0.17567833865594257
